<a href="https://colab.research.google.com/github/Sunflower47/Portfolio/blob/main/statistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание

Вы должны реализовать следующие алгоритмы в файле mixfit.py:



Метод максимального правдоподобия для смеси двух одномерных нормальных распределений. Напишите функцию max_likelihood(x, tau, mu1, sigma1, mu2, sigma2, rtol=1e-3), где x — массив данных, остальные позиционные аргументы — начальные приближения для искомых параметров распределения, rtol — относительная точность поиска параметров, функция должна возвращать кортеж из параметров распределения в том же порядке, что и в сигнатуре функции. Для оптимизации разрешается использовать scipy.optimize.



EM-метод для смеси трех нормальных распределений — τ1 N(µ1, σ) + τ2 N(µ2, σ) + (1-τ1-τ2) N(0, σ0). Напишите функцию em_double_cluster(x, tau1, tau2, muv, mu1, mu2, sigma02, sigmax2, sigmav2, rtol=1e-5), где

 - x — массив (N, 4) состоящий из двух столбцов координат звезд, и двух столбцов собственых движений звезд. Собственными движениями называется скорость движения звезды в картинной плоскости в угловых единицах;
τ1 (tau1), τ2 (tau2) — относительные количества звезд в первом и втором скоплениях. Обратите внимание, что относительное количество звезд поля может быть найдена из условия нормировки (1-τ1-τ2);
 - µ1 — четырехмерный параметр распределения звезд в первом скоплении. Состоит из среднего положения звезд скопления mu1 и среднего собственного движения скопления muv;
 - µ2 — четырехмерный параметр распределения звезд во втором скоплении. Состоит из среднего положения звезд скопления mu2 и среднего собственного движения скопления muv. Обратите внимание, что предполагается, что скорость движения скоплений — одинаковая;
 - σ — диагональная матрица 4x4. Первые два компоненты которой равны sigmax2 и отвечают за разброс положения звезд скоплений вокруг среднего, а последние два компонента sigmav2 отвечают за разброс собственных движений звезд скоплений относительно среднего. Иначе говоря, предполагается, что разброс по координатам имеет симметричный характер и одинаковую дисперсию по обоим направлениям. Аналогичное предположение действует для разброса проекций скоростей.
 - σ0 — диагональная матрица 2x2, элементы которой равны sigma02. Разброс собственных движений звезд поля. Обратите внимание, что предполагается, что среднее собственное движение звезд поля — нулевое.

#Код

Импорт библиотек

In [7]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt


Нормальное распределение

In [8]:
def normal(x, mu, sigma):
    return 1/(2*np.pi*sigma) * np.exp(-(x-mu)**2 / (2*sigma**2))

Задание смеси распределений

In [9]:
def p(x, tau, mu1, sigma1, mu2, sigma2):
    return tau*normal(x, mu1, sigma1) + (1 - tau)*normal(x, mu2, sigma2)

Функция правдоподобия

In [10]:
def func_likehood(y, x):
    tau = y[0]
    mu1 = y[1]
    sigma1 = y[2]
    mu2 = y[3]
    sigma2 = y[4]
    return - np.sum(np.log(p(x, tau, mu1, sigma1, mu2, sigma2)))

##Задание функции максимального правдоподобия

In [11]:
def max_likelihood(x, tau, mu1, sigma1, mu2, sigma2, rtol=1e-3):
    return optimize.minimize(func_likehood, [tau, mu1, sigma1, mu2, sigma2], args = x,
                             bounds=[(0,1), (-500, 500), (-500, 500), (-500, 500), (-500, 500)], 
                             tol=rtol).x

Двумерное нормальное распределение

In [12]:
def m_double_gauss(x, tau, mu1, sigma1, mu2, sigma2, rtol=1e-3):
    y = np.array([tau, mu1, sigma1, mu2, sigma2])
    while True:
        tau = np.sum(tau*normal(x, mu1, sigma1))/ np.sum(p(x, tau, mu1, sigma1, mu2, sigma2))
        mu1 = np.sum(x*normal(x, mu1, sigma1))/np.sum(normal(x, mu1, sigma1))
        mu2 = np.sum(x*normal(x, mu2, sigma2))/np.sum(normal(x, mu2, sigma2))
        sigma1 = np.sqrt(np.sum((x-sigma1)**2 *normal(x, mu1, sigma1))/ np.sum(x*normal(x, mu1, sigma1)))      
        sigma2 = np.sqrt(np.sum((x-sigma2)**2 *normal(x, mu2, sigma2))/ np.sum(x*normal(x, mu2, sigma2)))
        y_new = np.array([tau, mu1, sigma1, mu2, sigma2])
        
        if np.linalg.norm(y - y_new) <= rtol:
            break
        y = y_new
        
    return y

##EM-алгоритм

In [13]:
def em_double_cluster(x, tau1, tau2, muv, mu1, mu2, sigma02, sigmax2, sigmav2, rtol=1e-5):
    sigma = np.concatenate((sigmax2,sigmav2))
    mu_1 = np.vstack((mu1, muv))
    mu_2 = np.vstack((mu2, muv))
    
    x_n1 = lambda mu_1,sigma, tau1: stats.multivariate_normal(mu_1, sigma).rvs(int(tau1))
    x_n2 = lambda mu_2,sigma, tau2:stats.multivariate_normal(mu_2, sigma).rvs(int(tau2))
    x_0  = lambda sigma02, tau1, tau2: stats.multivariate_normal(cov=sigma02).rvs(int(1 - tau1 - tau2))
    
    p = lambda tau1, tau2, muv, mu1, mu2, sigma, sigma02: x_0(mu_1,sigma, tau1)+ x_n1(mu_2,sigma, tau2) + x_n2(sigma02, tau1, tau2)
    
    while True:
        
        tau1 = np.sum(x_n1(mu_1,sigma, tau1))/p(tau1, tau2, muv, mu1, mu2, sigma02, sigmax2, sigmav2)
        tau2 = np.sum(x_n2(mu_2,sigma, tau2))/p(tau1, tau2, muv, mu1, mu2, sigma02, sigmax2, sigmav2)
        
        mu_1 = np.sum(x@x_n1(mu_1,sigma, tau1))/np.sum(x_n1(mu_1,sigma, tau1))
        mu_2 = np.sum(x@x_n1(mu_2,sigma, tau2))/np.sum(x_n2(mu_2,sigma, tau2))
        
        sigma = np.sum((x - mu_1).T @ x_n1(mu_1,sigma, tau1) @ (x - mu_1))/np.sum(x_n1(mu_1,sigma, tau1))
        
        sigma02 = np.sum(x.T@x_n2(sigma02, tau1, tau2)@x) / np.sum(x_n2(sigma02, tau1, tau2))
     
    
    
    return [tau1, tau2, muv, mu1, mu2, sigma[0:3], sigma[3:], sigma02]